# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json 

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
input_data_file = "weather_data_file.csv"
weather_data_df = pd.read_csv(input_data_file)
weather_data_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,Hong Kong,22.29,114.16,78.01,78,85,4.25,HK,1587340010
1,Cidreira,-30.18,-50.21,68.07,85,0,16.93,BR,1587340226
2,New Norfolk,-42.78,147.06,57.00,91,100,7.74,AU,1587340226
3,Kapaa,22.08,-159.32,80.60,61,1,10.29,US,1587340226
4,Kahului,20.89,-156.47,82.40,51,40,9.17,US,1587340227
...,...,...,...,...,...,...,...,...,...
577,Taganak,6.08,118.30,80.60,88,20,2.24,PH,1587340356
578,Mastic Beach,40.77,-72.85,50.00,80,79,5.01,US,1587340209
579,Savonlinna,61.87,28.88,37.99,74,20,6.93,FI,1587340357
580,Gorontalo,0.54,123.06,81.70,79,80,1.19,ID,1587340357


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_data_df[['Lat','Lng']]
humidity = weather_data_df['Humidity']

fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=humidity))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#utilize loc function

ideal_temp = weather_data_df.loc[(weather_data_df['Max Temp']<80) & (weather_data_df['Max Temp']>70)]
ideal_temp

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,Hong Kong,22.29,114.16,78.01,78,85,4.25,HK,1587340010
20,Arraial do Cabo,-22.97,-42.03,73.67,70,96,4.79,BR,1587340230
23,Rikitea,-23.12,-134.97,77.07,67,100,5.99,PF,1587340231
30,Buin,-33.73,-70.75,77.00,50,0,9.17,CL,1587340232
33,Souillac,-20.52,57.52,75.20,88,40,11.41,MU,1587340233
...,...,...,...,...,...,...,...,...,...
555,Santa Isabel Rio Negro,-0.41,-65.02,76.35,85,59,0.81,BR,1587340352
560,Quatre Cocos,-20.21,57.76,75.20,88,40,11.41,MU,1587340305
561,Cockburn Town,21.46,-71.14,79.18,82,7,16.49,TC,1587340230
562,Aswān,24.09,32.91,70.50,30,0,8.34,EG,1587340085


In [6]:
ideal_wind = ideal_temp.loc[(ideal_temp['Windspeed']<10)]
ideal_wind

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,Hong Kong,22.29,114.16,78.01,78,85,4.25,HK,1587340010
20,Arraial do Cabo,-22.97,-42.03,73.67,70,96,4.79,BR,1587340230
23,Rikitea,-23.12,-134.97,77.07,67,100,5.99,PF,1587340231
30,Buin,-33.73,-70.75,77.00,50,0,9.17,CL,1587340232
36,Atar,20.52,-13.05,71.76,20,0,8.34,MR,1587339970
...,...,...,...,...,...,...,...,...,...
546,Āsosa,10.07,34.53,72.16,18,98,1.19,ET,1587340349
552,Raja,8.46,25.68,73.22,20,70,4.85,SS,1587340351
555,Santa Isabel Rio Negro,-0.41,-65.02,76.35,85,59,0.81,BR,1587340352
562,Aswān,24.09,32.91,70.50,30,0,8.34,EG,1587340085


In [7]:
#i got 0 data for Cloudiness 
ideal_clouds = ideal_wind.loc[(ideal_wind['Cloudiness']==0)]
ideal_clouds

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
30,Buin,-33.73,-70.75,77.00,50,0,9.17,CL,1587340232
36,Atar,20.52,-13.05,71.76,20,0,8.34,MR,1587339970
86,Villamontes,-21.25,-63.50,71.82,34,0,1.54,BO,1587340246
266,Santa Cruz de la Sierra,-17.80,-63.17,77.00,65,0,2.24,BO,1587340286
291,Ballina,-28.87,153.57,71.60,56,0,8.05,AU,1587340291
320,Tikrit,34.61,43.68,71.47,45,0,3.00,IQ,1587340298
330,Adrar,20.50,-10.07,75.76,18,0,6.55,MR,1587340300
491,Lagunas,-5.23,-75.68,75.94,63,0,2.15,PE,1587340337
505,Taoudenni,22.68,-3.98,77.97,21,0,6.42,ML,1587340340
562,Aswān,24.09,32.91,70.50,30,0,8.34,EG,1587340085


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#Store into variable named hotel_df.
hotel_df = ideal_clouds
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
30,Buin,-33.73,-70.75,77.00,50,0,9.17,CL,1587340232
36,Atar,20.52,-13.05,71.76,20,0,8.34,MR,1587339970
86,Villamontes,-21.25,-63.50,71.82,34,0,1.54,BO,1587340246
266,Santa Cruz de la Sierra,-17.80,-63.17,77.00,65,0,2.24,BO,1587340286
291,Ballina,-28.87,153.57,71.60,56,0,8.05,AU,1587340291
320,Tikrit,34.61,43.68,71.47,45,0,3.00,IQ,1587340298
330,Adrar,20.50,-10.07,75.76,18,0,6.55,MR,1587340300
491,Lagunas,-5.23,-75.68,75.94,63,0,2.15,PE,1587340337
505,Taoudenni,22.68,-3.98,77.97,21,0,6.42,ML,1587340340
562,Aswān,24.09,32.91,70.50,30,0,8.34,EG,1587340085


In [9]:
#Add a "Hotel Name" column to the DataFrame.
hotel_df['Hotel Name'] = "" 
hotel_df

C:\Users\zoeyc\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
30,Buin,-33.73,-70.75,77.00,50,0,9.17,CL,1587340232,
36,Atar,20.52,-13.05,71.76,20,0,8.34,MR,1587339970,
86,Villamontes,-21.25,-63.50,71.82,34,0,1.54,BO,1587340246,
266,Santa Cruz de la Sierra,-17.80,-63.17,77.00,65,0,2.24,BO,1587340286,
291,Ballina,-28.87,153.57,71.60,56,0,8.05,AU,1587340291,
320,Tikrit,34.61,43.68,71.47,45,0,3.00,IQ,1587340298,
330,Adrar,20.50,-10.07,75.76,18,0,6.55,MR,1587340300,
491,Lagunas,-5.23,-75.68,75.94,63,0,2.15,PE,1587340337,
505,Taoudenni,22.68,-3.98,77.97,21,0,6.42,ML,1587340340,
562,Aswān,24.09,32.91,70.50,30,0,8.34,EG,1587340085,


In [14]:
#reindex for the for loop
#hotel_df.reset_index()
hotel_df.index=range(len(hotel_df))
hotel_df
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
0,Buin,-33.73,-70.75,77.00,50,0,9.17,CL,1587340232,
1,Atar,20.52,-13.05,71.76,20,0,8.34,MR,1587339970,
2,Villamontes,-21.25,-63.50,71.82,34,0,1.54,BO,1587340246,
3,Santa Cruz de la Sierra,-17.80,-63.17,77.00,65,0,2.24,BO,1587340286,
4,Ballina,-28.87,153.57,71.60,56,0,8.05,AU,1587340291,
5,Tikrit,34.61,43.68,71.47,45,0,3.00,IQ,1587340298,
6,Adrar,20.50,-10.07,75.76,18,0,6.55,MR,1587340300,
7,Lagunas,-5.23,-75.68,75.94,63,0,2.15,PE,1587340337,
8,Taoudenni,22.68,-3.98,77.97,21,0,6.42,ML,1587340340,
9,Aswān,24.09,32.91,70.50,30,0,8.34,EG,1587340085,Nuba Nile


In [15]:
#Set parameters to search for hotels with 5000 meters.
target_radius = 5000
target_type = 'lodging'

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for i in hotel_df.index:
    target_coordinates = hotel_df['Lat'][i].astype(str) + "," + hotel_df['Lng'][i].astype(str)



    params = {
      "location": target_coordinates,
      "radius": target_radius,
      "type": target_type,
      "key": g_key
    }


    response = requests.get(base_url, params=params)
    locations = response.json()

hotel_df['Hotel Name'][i] = locations["results"][0]["name"]

C:\Users\zoeyc\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
0,Buin,-33.73,-70.75,77.00,50,0,9.17,CL,1587340232,
1,Atar,20.52,-13.05,71.76,20,0,8.34,MR,1587339970,
2,Villamontes,-21.25,-63.50,71.82,34,0,1.54,BO,1587340246,
3,Santa Cruz de la Sierra,-17.80,-63.17,77.00,65,0,2.24,BO,1587340286,
4,Ballina,-28.87,153.57,71.60,56,0,8.05,AU,1587340291,
5,Tikrit,34.61,43.68,71.47,45,0,3.00,IQ,1587340298,
6,Adrar,20.50,-10.07,75.76,18,0,6.55,MR,1587340300,
7,Lagunas,-5.23,-75.68,75.94,63,0,2.15,PE,1587340337,
8,Taoudenni,22.68,-3.98,77.97,21,0,6.42,ML,1587340340,
9,Aswān,24.09,32.91,70.50,30,0,8.34,EG,1587340085,Nuba Nile


In [18]:
# Add marker layer ontop of heat map

humidity = hotel_df['Humidity']

fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=humidity, max_intensity = 250))
fig.add_layer(gmaps.marker_layer(locations, info_box_content=hotel_info))

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))